In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Conv2DTranspose, BatchNormalization, Activation, Add, MaxPooling2D, Concatenate
from tensorflow.keras.models import Model

# Residual Block
def residual_block(x, filters, downsample=False):
    shortcut = x
    strides = (2, 2) if downsample else (1, 1)

    x = Conv2D(filters, (3, 3), strides=strides, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)

    # Adjust shortcut if downsampling
    if downsample:
        shortcut = Conv2D(filters, (1, 1), strides=(2, 2), padding="same")(shortcut)
        shortcut = BatchNormalization()(shortcut)

    x = Add()([shortcut, x])
    x = Activation("relu")(x)
    return x

# Atrous Convolution (Dilated Convolution) Block
def atrous_block(x, filters):
    conv1 = Conv2D(filters, (3, 3), dilation_rate=1, padding="same", activation='relu')(x)
    conv2 = Conv2D(filters, (3, 3), dilation_rate=3, padding="same", activation='relu')(x)
    conv3 = Conv2D(filters, (3, 3), dilation_rate=5, padding="same", activation='relu')(x)
    concat = Concatenate()([conv1, conv2, conv3])
    output = Conv2D(filters, (1, 1), activation='relu')(concat)
    return output

# Multi-Kernel Pooling Block
def mkp_block(x):
    pool1 = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(x)
    pool2 = MaxPooling2D(pool_size=(3, 3), strides=1, padding='same')(x)
    pool3 = MaxPooling2D(pool_size=(5, 5), strides=1, padding='same')(x)
    pool4 = MaxPooling2D(pool_size=(6, 6), strides=1, padding='same')(x)

    concat = Concatenate()([pool1, pool2, pool3, pool4])
    output = Conv2D(256, (1, 1), activation='relu')(concat)
    return output

# Decoder Block
def decoder_block(x, skip, filters):
    x = Conv2DTranspose(filters, (3, 3), strides=(2, 2), padding="same")(x)
    x = Add()([x, skip])
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

# Full Model Assembly
def build_proposed_model(input_shape=(448, 448, 1)):
    inputs = Input(input_shape)

    # Initial Convolution
    x = Conv2D(64, (7, 7), strides=(2, 2), padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    # Encoder
    e1 = residual_block(x, 64, downsample=True)
    e1 = residual_block(e1, 64)
    e2 = residual_block(e1, 128, downsample=True)
    e2 = residual_block(e2, 128)
    e3 = residual_block(e2, 256, downsample=True)
    e3 = residual_block(e3, 256)

    # Atrous Convolution Block
    ac = atrous_block(e3, 256)

    # Multi Kernel Pooling Block
    mkp = mkp_block(ac)

    # Decoder
    d1 = decoder_block(mkp, e3, 256)
    d2 = decoder_block(d1, e2, 128)
    d3 = decoder_block(d2, e1, 64)
    d4 = Conv2DTranspose(64, (3, 3), strides=(2, 2), padding="same")(d3)

    # Final Output
    output = Conv2D(1, (1, 1), activation='sigmoid')(d4)

    model = Model(inputs, output)
    return model

# Instantiate and compile model
model = build_proposed_model()
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()
